<a href="https://colab.research.google.com/github/Bhavani-Rajan/DS-Unit-2-Regression-Classification/blob/master/module3/assignment_regression_classification_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 3

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`). 

Use a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.** 

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.

- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Fit a ridge regression model with multiple features.
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.


## Stretch Goals
- [ ] Add your own stretch goal(s) !
- [ ] Instead of `RidgeRegression`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `RidgeRegression`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [1]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module3')

Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 156, done.
remote: Total 156 (delta 0), reused 0 (delta 0), pack-reused 156
Receiving objects: 100% (156/156), 19.30 MiB | 16.35 MiB/s, done.
Resolving deltas: 100% (71/71), done.
From https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> origin/master
     |████████████████████████████████| 92kB 5.9MB/s 
     |████████████████████████████████| 112kB 16.4MB/s 
     |████████████████████████████████| 133kB 59.3MB/s 
     |████████████████████████████████| 57.7MB 1.3MB/s 
     |████████████████████████████████| 235kB 52.1MB/s 
     |████████████████████████████████| 614kB 50.2MB/s 
     |████████████████████████████████| 235kB 54.1MB/s 
     |████████████████████████████████| 307kB 45.9MB/s 
     |████████████████████████████████| 51kB 26.8MB/s 
     |████████████████████████████████| 215kB 48.9MB/s 
     |█████████

In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv('../data/condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
BLOCK,23040.0,4.459116e+03,3.713270e+03,1.0,1341.0,3546.0,6673.75,16350.0
LOT,23040.0,3.539485e+02,6.280253e+02,1.0,22.0,49.0,375.00,9057.0
EASE-MENT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZIP_CODE,23039.0,1.079119e+04,1.103414e+03,0.0,10306.0,11211.0,11360.00,11697.0
RESIDENTIAL_UNITS,23039.0,1.827423e+00,1.009043e+01,0.0,0.0,1.0,2.00,750.0
COMMERCIAL_UNITS,23039.0,2.865142e-01,5.778224e+00,-148.0,0.0,0.0,0.00,570.0
TOTAL_UNITS,23039.0,2.320891e+00,1.207055e+01,0.0,1.0,1.0,2.00,755.0
GROSS_SQUARE_FEET,23039.0,3.508555e+03,2.218610e+04,0.0,494.0,1356.0,2280.00,1303935.0
YEAR_BUILT,23005.0,1.819672e+03,4.883766e+02,0.0,1920.0,1940.0,1965.00,2019.0
TAX_CLASS_AT_TIME_OF_SALE,23040.0,1.624479e+00,8.038978e-01,1.0,1.0,1.0,2.00,4.0


## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`). 

Use a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.** 

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.

- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Fit a ridge regression model with multiple features.
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.




In [7]:
df.dtypes

BOROUGH                            object
NEIGHBORHOOD                       object
BUILDING_CLASS_CATEGORY            object
TAX_CLASS_AT_PRESENT               object
BLOCK                               int64
LOT                                 int64
EASE-MENT                         float64
BUILDING_CLASS_AT_PRESENT          object
ADDRESS                            object
APARTMENT_NUMBER                   object
ZIP_CODE                          float64
RESIDENTIAL_UNITS                 float64
COMMERCIAL_UNITS                  float64
TOTAL_UNITS                       float64
LAND_SQUARE_FEET                   object
GROSS_SQUARE_FEET                 float64
YEAR_BUILT                        float64
TAX_CLASS_AT_TIME_OF_SALE           int64
BUILDING_CLASS_AT_TIME_OF_SALE     object
SALE_PRICE                          int64
SALE_DATE                          object
dtype: object

In [8]:
df.shape

(23040, 21)

#### Use a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.*

In [9]:
df_new = df[(df['SALE_PRICE'] >= 100000) & (df['SALE_PRICE'] <= 2000000 ) ]
df_new.shape

(13471, 21)

#### for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`). 

In [10]:
df_new = df_new[(df_new['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS')]
df_new.shape

(3164, 21)

#### Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.

In [11]:
df_new['SALE_DATE'] = pd.to_datetime(df_new['SALE_DATE'], infer_datetime_format=True)

df_new['SALE_DATE'].dt.month.value_counts()

1    952
3    802
2    763
4    647
Name: SALE_DATE, dtype: int64

In [0]:
# Do train/test split
# Use data from Jan - Mar 2019 to train
# Use data from Apr 2019 to test
df_new['SALE_DATE'] = pd.to_datetime(df_new['SALE_DATE'], infer_datetime_format=True)
cutoff = pd.to_datetime('2019-04-01')
train = df_new[df_new.SALE_DATE < cutoff]
test  = df_new[df_new.SALE_DATE >= cutoff]

In [13]:
train.shape, test.shape

((2517, 21), (647, 21))

####  Do one-hot encoding of categorical features.

In [14]:
train.select_dtypes(include='number').describe().T

,count,mean,std,min,25%,50%,75%,max
BLOCK,2517.0,6750.859754,3977.126707,21.0,3818.0,6021.0,9879.0,16350.0
LOT,2517.0,75.674215,157.232723,1.0,21.0,42.0,70.0,2720.0
EASE-MENT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZIP_CODE,2517.0,10993.891538,494.054748,10301.0,10314.0,11234.0,11413.0,11697.0
RESIDENTIAL_UNITS,2517.0,0.987684,0.110315,0.0,1.0,1.0,1.0,1.0
COMMERCIAL_UNITS,2517.0,0.016289,0.129712,0.0,0.0,0.0,0.0,2.0
TOTAL_UNITS,2517.0,1.003973,0.171453,0.0,1.0,1.0,1.0,3.0
GROSS_SQUARE_FEET,2517.0,1473.500596,599.132016,0.0,1144.0,1367.0,1683.0,7875.0
YEAR_BUILT,2517.0,1944.729837,27.054972,1890.0,1925.0,1940.0,1960.0,2018.0
TAX_CLASS_AT_TIME_OF_SALE,2517.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0


In [15]:
train.select_dtypes(exclude='number').describe().T

,count,unique,top,freq,first,last
BOROUGH,2517,5,4,1209,NaN,NaN
NEIGHBORHOOD,2517,6,OTHER,2390,NaN,NaN
BUILDING_CLASS_CATEGORY,2517,1,01 ONE FAMILY DWELLINGS,2517,NaN,NaN
TAX_CLASS_AT_PRESENT,2517,2,1,2486,NaN,NaN
BUILDING_CLASS_AT_PRESENT,2517,13,A1,921,NaN,NaN
ADDRESS,2517,2507,130-52 LEFFERTS BOULEVARD,2,NaN,NaN
APARTMENT_NUMBER,1,1,RP.,1,NaN,NaN
LAND_SQUARE_FEET,2517,889,"4,000",235,NaN,NaN
BUILDING_CLASS_AT_TIME_OF_SALE,2517,11,A1,921,NaN,NaN
SALE_DATE,2517,68,2019-01-31 00:00:00,78,2019-01-01 00:00:00,2019-03-30 00:00:00


In [0]:
target = 'SALE_PRICE'
high_cardinality = ['BOROUGH','NEIGHBORHOOD', 'BUILDING_CLASS_AT_PRESENT',
                    'ADDRESS', 'APARTMENT_NUMBER','LAND_SQUARE_FEET',
                    'BUILDING_CLASS_AT_TIME_OF_SALE',
                    'SALE_DATE','EASE-MENT']
features = train.columns.drop([target] + high_cardinality)

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [17]:
X_train.head()

,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
44,01 ONE FAMILY DWELLINGS,1,5495,801,11230.0,1.0,0.0,1.0,1325.0,1930.0,1
61,01 ONE FAMILY DWELLINGS,1,7918,72,11427.0,1.0,0.0,1.0,2001.0,1940.0,1
78,01 ONE FAMILY DWELLINGS,1,4210,19,10461.0,1.0,0.0,1.0,2043.0,1925.0,1
108,01 ONE FAMILY DWELLINGS,1,5212,69,11226.0,1.0,0.0,1.0,2680.0,1899.0,1
111,01 ONE FAMILY DWELLINGS,1,7930,121,11203.0,1.0,0.0,1.0,1872.0,1940.0,1


In [18]:
train.groupby('TAX_CLASS_AT_PRESENT')['SALE_PRICE'].mean()

TAX_CLASS_AT_PRESENT
1     623855.781175
1D    392900.000000
Name: SALE_PRICE, dtype: float64

In [0]:
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [20]:
X_train.head()

,BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
44,1,1,0,5495,801,11230.0,1.0,0.0,1.0,1325.0,1930.0,1
61,1,1,0,7918,72,11427.0,1.0,0.0,1.0,2001.0,1940.0,1
78,1,1,0,4210,19,10461.0,1.0,0.0,1.0,2043.0,1925.0,1
108,1,1,0,5212,69,11226.0,1.0,0.0,1.0,2680.0,1899.0,1
111,1,1,0,7930,121,11203.0,1.0,0.0,1.0,1872.0,1940.0,1


#### Do feature selection with SelectKBest.

In [21]:
# How many features do we have currently?
features = X_train.columns
n = len(features)
n

12

In [22]:
X_train.isnull().sum()

BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS    0
TAX_CLASS_AT_PRESENT_1                             0
TAX_CLASS_AT_PRESENT_1D                            0
BLOCK                                              0
LOT                                                0
ZIP_CODE                                           0
RESIDENTIAL_UNITS                                  0
COMMERCIAL_UNITS                                   0
TOTAL_UNITS                                        0
GROSS_SQUARE_FEET                                  0
YEAR_BUILT                                         0
TAX_CLASS_AT_TIME_OF_SALE                          0
dtype: int64

In [23]:
# Select the 10 features that best correlate with the target
# (4 is an arbitrary starting point here)
from sklearn.feature_selection import f_regression, SelectKBest

# Similar API to what we've seen before
selector = SelectKBest(score_func=f_regression, k=4)

# IMPORTANT!
# .fit_transform on the train set
# .transform on test set
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)
X_train_selected.shape, X_test_selected.shape

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


((2517, 4), (647, 4))

In [24]:
# Which features were selected?
all_names = X_train.columns
selected_mask = selector.get_support()
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)
    
print('\n')
print('Features not selected:')
for name in unselected_names:
    print(name)

Features selected:
BLOCK
ZIP_CODE
TOTAL_UNITS
GROSS_SQUARE_FEET


Features not selected:
BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS
TAX_CLASS_AT_PRESENT_1
TAX_CLASS_AT_PRESENT_1D
LOT
RESIDENTIAL_UNITS
COMMERCIAL_UNITS
YEAR_BUILT
TAX_CLASS_AT_TIME_OF_SALE


In [25]:
# How many features should be selected?
# You can try a range of values for k,
# then choose the model with the best score.
# If multiple models "tie" for the best score,
# choose the simplest model.
# You decide what counts as a tie!

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

for k in range(1, len(X_train.columns)+1):
    
    print(f'{k} features')
    
    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)
    
    model = LinearRegression()
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test MAE: ${mae:,.0f} \n')

1 features
Test MAE: $185,788 

2 features
Test MAE: $185,954 

3 features
Test MAE: $169,011 

4 features
Test MAE: $169,002 

5 features
Test MAE: $170,068 

6 features
Test MAE: $170,068 

7 features
Test MAE: $169,930 

8 features
Test MAE: $169,871 

9 features
Test MAE: $169,876 

10 features
Test MAE: $169,666 

11 features
Test MAE: $169,666 

12 features
Test MAE: $169,666 



/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in

## checking at the errors, 4 features are the best to predict, according to kbest selector,where the error is lowest and rises after by the increase of features.

In [26]:
# Correlation matrix!
X_train.corr()

,BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TAX_CLASS_AT_PRESENT_1,NaN,1.000000,-1.000000,-0.171270,-0.104195,-0.069107,1.000000,0.014026,0.654025,0.274691,0.048690,NaN
TAX_CLASS_AT_PRESENT_1D,NaN,-1.000000,1.000000,0.171270,0.104195,0.069107,-1.000000,-0.014026,-0.654025,-0.274691,-0.048690,NaN
BLOCK,NaN,-0.171270,0.171270,1.000000,0.048942,0.643294,-0.171270,-0.027223,-0.130793,-0.132616,-0.186660,NaN
LOT,NaN,-0.104195,0.104195,0.048942,1.000000,-0.037637,-0.104195,-0.030453,-0.090079,-0.020997,0.050542,NaN
ZIP_CODE,NaN,-0.069107,0.069107,0.643294,-0.037637,1.000000,-0.069107,0.045575,-0.009984,-0.071140,-0.432685,NaN
RESIDENTIAL_UNITS,NaN,1.000000,-1.000000,-0.171270,-0.104195,-0.069107,1.000000,0.014026,0.654025,0.274691,0.048690,NaN
COMMERCIAL_UNITS,NaN,0.014026,-0.014026,-0.027223,-0.030453,0.045575,0.014026,1.000000,0.765572,0.155590,-0.091842,NaN
TOTAL_UNITS,NaN,0.654025,-0.654025,-0.130793,-0.090079,-0.009984,0.654025,0.765572,1.000000,0.294451,-0.038155,NaN
GROSS_SQUARE_FEET,NaN,0.274691,-0.274691,-0.132616,-0.020997,-0.071140,0.274691,0.155590,0.294451,1.000000,0.130204,NaN


#### Do feature scaling.

In [27]:
from sklearn.preprocessing import StandardScaler


# standardize the features
scaler = StandardScaler().fit(X_train_selected)
rescaled_X_train_selected = scaler.transform(X_train_selected)


rescaled_X_train_selected

array([[ 0.        ,  0.1116684 , -0.1116684 , ..., -0.24790881,
        -0.54454923,  0.        ],
       [ 0.        ,  0.1116684 , -0.1116684 , ...,  0.8806143 ,
        -0.17485795,  0.        ],
       [ 0.        ,  0.1116684 , -0.1116684 , ...,  0.95072964,
        -0.72939487,  0.        ],
       ...,
       [ 0.        ,  0.1116684 , -0.1116684 , ...,  0.5567482 ,
         2.70873401,  0.        ],
       [ 0.        ,  0.1116684 , -0.1116684 , ..., -1.42317548,
        -0.54454923,  0.        ],
       [ 0.        ,  0.1116684 , -0.1116684 , ..., -0.5183537 ,
         0.19483333,  0.        ]])

#### Fit a ridge regression model with multiple features.

In [38]:
# for all features
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error

ridge_reg = Ridge().fit(X_train, y_train)
mean_absolute_error(y_test, ridge_reg.predict(X_test))

169640.54187801987

In [33]:
# for kbest mean selected features
# Let's try split

ridge_reg_split = Ridge().fit(X_train_selected, y_train)
mean_absolute_error(y_pred, ridge_reg_split.predict(X_test_selected))

116.78175888814096

####  Get mean absolute error for the test set.